In [1]:
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions
from beam_postgres import splitters
from beam_postgres.io import ReadFromPostgres
from datetime import datetime

class FilterWeekends(beam.DoFn):
    def process(self, element):
        # Get the datetime from the element
        dt = element['datetime']
        
        # Check if the day of the week is a weekend (5 or 6 are Saturday and Sunday)
        if dt.weekday() >= 5:
            yield element

with beam.Pipeline(options=PipelineOptions()) as p:
    read_from_postgres = ReadFromPostgres(
            query="SELECT * FROM public.your_table_name;",
            host="localhost",
            database="transport",
            user="postgres",
            password="postgres",
            splitter=splitters.NoSplitter()  # you can select how to split query for performance
    )

    # Read data from PostgreSQL and store in a PCollection named 'data'
    data = p | "ReadFromPostgres" >> read_from_postgres

    # Use the 'data' PCollection and filter for weekend data
    weekend_data = data | "FilterWeekends" >> beam.ParDo(FilterWeekends())

    # Output weekend data to stdout or other downstream steps
    (
        weekend_data
        #| "WriteToStdout" >> beam.Map(print)
        | "Write to file" >> beam.io.WriteToText('data/to_hive1')
        # Add more pipeline steps here
    )


INFO:beam_postgres.client:Successfully execute query: EXPLAIN SELECT * FROM (SELECT * FROM public.your_table_name) as subq
INFO:beam_postgres.client:Successfully execute query: SELECT * FROM public.your_table_name


In [2]:
import csv
import datetime

input_file = "data/to_hive1-00000-of-00001"
csv_file = 'output1.csv'

with open(input_file, 'r') as file:
    lines = file.readlines()

data = [eval(line) for line in lines if line.strip() != 'None']

if data:
    with open(csv_file, mode='w', newline='') as file:
        fieldnames = data[0].keys()
        writer = csv.DictWriter(file, fieldnames=fieldnames)

        writer.writeheader()
        for row in data:
            writer.writerow(row)

    print(f'CSV file "{csv_file}" has been created.')
else:
    print('No valid data found in the input file.')


CSV file "output1.csv" has been created.
